In [ ]:
!pip install transformers
!pip install pytube

In [ ]:
!git clone https://github.com/abreza/clip-video-embedder
%cd clip-video-embedder/

In [1]:
from utils.video_loader import download_video_from_youtube

video_id = "4uw4co69JUQ"
video_path = download_video_from_youtube('4uw4co69JUQ', './videos')

In [ ]:
from transformers import CLIPModel, CLIPProcessor

model = CLIPModel.from_pretrained("clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("clip-vit-large-patch14")

In [ ]:
import numpy as np

from modules.frame_sampler.uniform_sampler import uniform_sample_frames
from modules.frame_sampler.aggregate_embedding import aggregate_embeddings

num_frames = 10
video_frames = uniform_sample_frames(video_path, num_frames)

frame_embeddings = []
for frame in video_frames:
    inputs = processor(images=frame, return_tensors="pt")
    outputs = model.get_image_features(**inputs)
    frame_embedding = outputs.detach().numpy()
    frame_embeddings.append(frame_embedding)

video_embedding = aggregate_embeddings(frame_embeddings, strategy="mean")

# Compare the video embedding to a given text
texts = ["a dog playing in the park", "playing basketball"]
for text in texts:
  text_inputs = processor(text=text, return_tensors="pt", padding=True)
  text_outputs = model.get_text_features(**text_inputs)
  text_embedding = text_outputs.detach().numpy()
  
  similarity = np.inner(video_embedding, text_embedding)
  print(text, similarity)